# 1. Information about the submission

## 1.1 Name and number of the assignment 

**Text Detoxification - HW3**



## 1.2 Student name

**Vo Ngoc Bich Uyen**

## 1.3 Codalab user ID

**UyenVo** 

## 1.4 Additional comments

***Enter here** any additional comments which you would like to communicate to a TA who is going to grade this work not related to the content of your submission.*

# 2. Technical Report

## 2.1 Methodology 

### 2.1.1 Dataset

+ train.tsv: 6,948 sentences with 1 to 3 detoxified versions. (neutral_comment1,	neutral_comment2,	neutral_comment3)
+ test.tsv: 875 source (toxic) sentences



### 2.1.2 Description

My approach is to mix the Delete and RuT5 Baseline methods. It first applies the delete method to remove toxic words from the input text using the tokenize function. Then, the paraphrasing method is applied to generate the paraphrased output.
1. Reading the input dataset
2. Preparing data structures for training: 
  
  Paired examples of toxic and neutral comments are created for training a model to distinguish between the two classes. The 'DetoxDataset' class handles the tokenization and preparation of the data for model input.
3. Defining the type of the model: 

  Initializing the fine-tuned ruT5-base model and tokenizer using the sberbank-ai/ruT5-base pre-trained model. We set the device to be either CUDA if available or CPU.
4. Training the model: 

  'Seq2SeqTrainingArguments' is initialized to define various training arguments and hyperparameters for the training process. Some notable arguments include:
  + batch sizes for training and evaluation (=8)
  + weight decay (=1e-5)
  + number of training epochs (=3)
  + learning rate (=1e-5)
  + logging step (=300)
  
  Then, performs the training process using the 'Seq2SeqTrainer' class. 

  Note: The untrained model produces better results than the trained model. I think that the model 'SkolkovoInstitute/ruT5-base-detox' may have a more specialized architecture for this specific task.

5. Generate detoxifications: 
+ The tokenize function for the delete method is defined.
+ The paraphrase function takes the input text, model, tokenizer, toxic words list, and other parameters.

## 2.2 Discussion of results


Method|Style transfer accuracy | Meaning preservation |	Fluency	| Joint score	| ChrF1
--- | --- | --- | --- | --- | --- 
delete| 0.56 | 0.89 | 0.85 | 0.41 | 0.53
ruT5base|0.79 | 0.83 | 0.84 | 0.56 | 0.57
delete+ruT5base|0.84 | 0.79 |0.76	| 0.52 | 0.54

Comparing the results of the different methods, we can make the following observations:

* Delete method: performs relatively well in terms of meaning preservation and fluency, but the style transfer accuracy and joint score are comparatively lower.

* ruT5-base method: outperforms the delete method in style transfer accuracy and joint score while maintaining a reasonable level of meaning preservation and fluency.

* delete + ruT5-base method: combining the delete method with the ruT5-base method yields improved results. While the style transfer accuracy remains high, the meaning preservation and fluency scores slightly decrease compared to the individual ruT5-base method. However, the joint score suggests that this combined method performs well across multiple metrics.

Overall, the ruT5-base method shows better performance than the delete method in terms of style transfer accuracy and joint score. However, combining the delete method with the ruT5-base method further improves the style transfer accuracy while maintaining a reasonable level of meaning preservation and fluency. 

Based on the results of three methods, I decided to choose the ruT5-base to submit to Codalab.


#3 Code

## 3.1 Requirements

In [ ]:
!git clone https://github.com/s-nlp/russe_detox_2022.git

Cloning into 'russe_detox_2022'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 81
Unpacking objects: 100% (82/82), 40.90 MiB | 16.76 MiB/s, done.


In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.9 MB/s eta 0:00:00


In [ ]:
!pip install ufal.udpipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 937.0/937.0 kB 16.9 MB/s eta 0:00:00


In [ ]:
import os
import re
import pandas as pd
import re

import torch
from tqdm import tqdm
from sklearn.utils import shuffle
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5ForConditionalGeneration,
    AutoTokenizer
)
from sklearn.model_selection import train_test_split
import gensim
from ufal.udpipe import Model, Pipeline

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## 3.2 Download the data

In [ ]:
df = pd.read_csv('/content/russe_detox_2022/data/input/train.tsv', sep='\t')
df = df.fillna('')
df.head()

,index,toxic_comment,neutral_comment1,neutral_comment2,neutral_comment3
0,0,"и,чё,блядь где этот херой был до этого со свои...","Ну и где этот герой был,со своими доказательст...",Где этот герой был до этого со своими доказате...,"и,где этот герой был до этого со своими доказа..."
1,1,"О, а есть деанон этого петуха?","О, а есть деанон",,
2,2,"херну всякую пишут,из-за этого лайка.долбоебизм.","Чушь всякую пишут, из- за этого лайка.","Ерунду всякую пишут,из-за этого лайка.",
3,3,из за таких пидоров мы и страдаем,из за таких плохих людей мы и страдаем,Из-за таких людей мы и страдаем,из за таких как он мы и страдаем
4,4,гондон путинский он а не артист,"Человек Путина он, а не артист",,


## 3.3 Preprocessing 

In [ ]:
with open('/content/russe_detox_2022/baselines/delete/toxic_vocab_extended.txt', 'r') as file:
    toxic_words = file.readlines()
toxic_words = [sentence.strip() for sentence in toxic_words]

In [ ]:
df_train_toxic = []
df_train_neutral = []

for index, row in df.iterrows():
    references = row[['neutral_comment1', 'neutral_comment2', 'neutral_comment3']].tolist()
    
    for reference in references:
        if len(reference) > 0:
            df_train_toxic.append(row['toxic_comment'])
            df_train_neutral.append(reference)
        else:
            break

In [ ]:
df = pd.DataFrame({
    'toxic_comment': df_train_toxic,
    'neutral_comment': df_train_neutral
})

df = shuffle(df)

In [ ]:
df.head()

,toxic_comment,neutral_comment
7163,"что не день,то хуйня(","Что не день, то плохой день("
3863,таким дэбилам сука такие машины!!! пешком нахуй,таким людям такие машины!!! пешком пусть ходят
2167,а этот манипулятор херов не думает когда шахтё...,"А этот манипулятор не думает,когда шахтеры нач..."
2640,"делайте уже, суки, хоть что-то кроме воровства.",Делайте хоть что нибудь. Перестаньте воровать
8160,"все мои одноклассники, вроде, получились норма...",Все мои одноклассники ки вроде получились норм...


In [ ]:
class DetoxDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        self.tokenizer.src_lang = "ru"
        self.tokenizer.tgt_lang = "ru"

    def __getitem__(self, idx):       

        source = self.tokenizer(
            self.data.iloc[idx].toxic_comment,
            max_length=150,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer(
            self.data.iloc[idx].neutral_comment,
            max_length=150,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        source["labels"] = target["input_ids"]

        return {k: v.squeeze(0) for k, v in source.items()}

    def __len__(self):
        return self.data.shape[0]

## 3.4 Modeling 

In [ ]:
base_model_name = 'sberbank-ai/ruT5-base'
model_name = 'SkolkovoInstitute/ruT5-base-detox'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
# train, val = train_test_split(df, random_state=42, test_size=0.01)
# trainset = DetoxDataset(train, tokenizer)
# valset = DetoxDataset(val, tokenizer)

In [ ]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="detox",
#     do_train=True,
#     do_eval=True,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8, 
#     weight_decay=1e-5,
#     num_train_epochs=3, 
#     learning_rate=1e-5,
#     evaluation_strategy="steps",
#     save_strategy="no",
#     save_total_limit=1,
#     logging_steps=300,
#     gradient_accumulation_steps=1,
# )

In [ ]:
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=trainset,
#     eval_dataset=valset,
#     tokenizer=tokenizer,
# )

In [ ]:
# trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
300,0.748900,0.062867
600,0.069800,0.062832
900,0.066200,0.063010
1200,0.066300,0.062955
1500,0.064500,0.062860
1800,0.064400,0.062764
2100,0.063000,0.062781
2400,0.064700,0.063301
2700,0.064800,0.062393
3000,0.060400,0.062834


TrainOutput(global_step=4119, training_loss=0.11385565737145247, metrics={'train_runtime': 2596.0466, 'train_samples_per_second': 12.687, 'train_steps_per_second': 1.587, 'total_flos': 5876146301184000.0, 'train_loss': 0.11385565737145247, 'epoch': 3.0})

## 3.5 Generate detoxifications

In [ ]:
df = pd.read_csv('/content/russe_detox_2022/data/input/test.tsv', sep='\t', encoding='utf-8')
toxic_inputs = df['toxic_comment'].tolist()

### Delete baseline

In [ ]:
with open('/content/russe_detox_2022/baselines/delete/toxic_vocab_extended.txt', 'r') as file:
    toxic_words = file.readlines()
toxic_words = [sentence.strip() for sentence in toxic_words]
modelfile = '/content/russe_detox_2022/baselines/delete/udpipe_syntagrus.model'
model_udpipe = Model.load(modelfile)
process_pipeline = Pipeline(model_udpipe, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

def tokenize(text, tags=False, lemmas=False):
    processed = process_pipeline.process(text)
    content = [l for l in processed.split('\n') if not l.startswith('#')]
    tagged = [w.split('\t') for w in content if w]

    tokens = []
    for token in tagged:
        if token[3] == 'PUNCT':
            continue

        token_res = ''
        if lemmas:
            token_res = token[2]
        else:
            token_res = token[1]
        if tags:
            token_res += '_' + token[3]
        tokens.append(token_res)

    return tokens

In [ ]:
results = []

for sample in tqdm(toxic_inputs):
    try:
        tokens_lemmas = tokenize(sample, lemmas=True)
    except:
        print(sample)
        tokens_lemmas = sample.split(' ')
    tokens = tokenize(sample, lemmas=False)
    cleaned_sentence = [tokens[i] for i, word in enumerate(tokens_lemmas) if word not in toxic_words]
        
    results.append(' '.join(cleaned_sentence))

100%|██████████| 875/875 [00:37<00:00, 23.31it/s]


In [ ]:
with open(f"/content/delete.txt", "w") as f:
    f.write("\n".join(results))

### ruT5 baseline

In [ ]:
def paraphrase(text, model, tokenizer, n=None, max_length='auto', temperature=0.0, beams=3):
    texts = [text] if isinstance(text, str) else text

    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        pad_token_id=50256,
        do_sample=False,
        temperature=0,
        repetition_penalty=1.0,
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [ ]:
preds = []
for text in tqdm(toxic_inputs):
    preds.append(paraphrase(text, model, tokenizer))

100%|██████████| 875/875 [05:28<00:00,  2.66it/s]


In [ ]:
with open(f"/content/t5_base.txt", "w") as f:
    f.write("\n".join(preds))

### Delete + ruT5 baseline

In [ ]:
def de_para(text, model, tokenizer, toxic_words, n=None, max_length='auto', temperature=0.0, beams=3):
    texts = [text] if isinstance(text, str) else text

    # Delete method
    for i, text in enumerate(texts):
        try:
            tokens_lemmas = tokenize(text, lemmas=True)
        except:
            print(text)
            tokens_lemmas = text.split(' ')
        tokens = tokenize(text, lemmas=False)
        cleaned_sentence = [tokens[i] for i, word in enumerate(tokens_lemmas) if word not in toxic_words]
        texts[i] = ' '.join(cleaned_sentence)

    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        pad_token_id=50256,
        do_sample=False,
        temperature=0,
        repetition_penalty=1.0,
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [ ]:
preds = []
for text in tqdm(toxic_inputs):
    preds.append(de_para(text, model, tokenizer, toxic_words))

100%|██████████| 875/875 [06:00<00:00,  2.43it/s]


In [ ]:
with open(f"/content/result.txt", "w") as f:
    f.write("\n".join(preds))